<a href="https://colab.research.google.com/github/asnoldy02-cell/sds510/blob/main/Module_4_Essentials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests pandas beautifulsoup4 mastodon.py lxml


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 5.7 MB/s eta 0:00:00


In [12]:
import requests
import pandas as pd
import os
import time
from datetime import datetime
from mastodon import Mastodon
from bs4 import BeautifulSoup
import random

In [3]:
KEYWORDS = [
    "medicine", "medical", "drug", "therapy", "treatment",
    "breakthrough", "research", "innovation", "health", "upcoming"
]
OUTPUT_FILE = "social_keyword_hits.csv"
HEADERS = {"User-Agent": "Mozilla/5.0 (compatible; OSINTMonitor/1.0)"}


In [5]:
def fetch_mastodon_posts(keywords):
    print("🐘 Fetching Mastodon posts...")
    results = []
    try:
        masto = Mastodon(api_base_url="https://mastodon.social")
        posts = masto.timeline_public(limit=80)
        for post in posts:
            content = BeautifulSoup(post["content"], "html.parser").get_text()
            for kw in keywords:
                if kw.lower() in content.lower():
                    results.append({
                        "source": "Mastodon",
                        "keyword": kw,
                        "content": content[:250],
                        "url": post["url"],
                        "timestamp": post["created_at"]
                    })
    except Exception as e:
        print(f"⚠️ Mastodon error: {e}")
    return results

In [14]:
def fetch_twitter_posts(keywords):
    print("🐦 Fetching Twitter (via Nitter)...")
    results = []
    for kw in keywords:
        url = f"https://nitter.net/search?f=tweets&q={kw}"
        try:
            resp = requests.get(url, headers=HEADERS, timeout=10)
            if resp.status_code == 200:
                soup = BeautifulSoup(resp.text, "lxml")
                tweets = soup.select("div.tweet-content")
                for t in tweets[:10]:
                    text = t.get_text(" ", strip=True)
                    if kw.lower() in text.lower():
                        results.append({
                            "source": "Twitter (Nitter)",
                            "keyword": kw,
                            "content": text[:250],
                            "url": url,
                            "timestamp": datetime.now().isoformat()
                        })
            else:
                print(f"⚠️ Twitter/Nitter search error {resp.status_code} for '{kw}'")
        except Exception as e:
            print(f"⚠️ Twitter error for '{kw}': {e}")
        time.sleep(2)
    return results


In [7]:
def fetch_telegram_posts(keywords):
    print("📱 Fetching Telegram posts...")
    results = []
    for kw in keywords:
        search_url = f"https://tgstat.com/en/search?query={kw}"
        try:
            resp = requests.get(search_url, headers=HEADERS, timeout=10)
            if resp.status_code == 200:
                soup = BeautifulSoup(resp.text, "lxml")
                posts = soup.select("div.card-body")
                for p in posts[:10]:
                    text = p.get_text(" ", strip=True)
                    link = p.find("a", href=True)
                    if kw.lower() in text.lower():
                        results.append({
                            "source": "Telegram",
                            "keyword": kw,
                            "content": text[:250],
                            "url": link["href"] if link else search_url,
                            "timestamp": datetime.now().isoformat()
                        })
            else:
                print(f"⚠️ Telegram search error {resp.status_code} for '{kw}'")
        except Exception as e:
            print(f"⚠️ Telegram error for '{kw}': {e}")
        time.sleep(2)
    return results

In [8]:
def fetch_parler_posts(keywords):
    print("🦜 Fetching Parler posts (DuckDuckGo)...")
    results = []
    for kw in keywords:
        search_url = f"https://duckduckgo.com/html/?q=site:parler.com {kw}"
        try:
            resp = requests.get(search_url, headers=HEADERS, timeout=10)
            soup = BeautifulSoup(resp.text, "lxml")
            links = soup.select("a.result__a")
            for l in links[:5]:
                text = l.get_text(" ", strip=True)
                if kw.lower() in text.lower():
                    results.append({
                        "source": "Parler",
                        "keyword": kw,
                        "content": text[:250],
                        "url": l["href"],
                        "timestamp": datetime.now().isoformat()
                    })
        except Exception as e:
            print(f"⚠️ Parler error: {e}")
        time.sleep(2)
    return results


In [9]:
def fetch_truthsocial_posts(keywords):
    print("🕊 Fetching Truth Social posts (DuckDuckGo)...")
    results = []
    for kw in keywords:
        search_url = f"https://duckduckgo.com/html/?q=site:truthsocial.com {kw}"
        try:
            resp = requests.get(search_url, headers=HEADERS, timeout=10)
            soup = BeautifulSoup(resp.text, "lxml")
            links = soup.select("a.result__a")
            for l in links[:5]:
                text = l.get_text(" ", strip=True)
                if kw.lower() in text.lower():
                    results.append({
                        "source": "Truth Social",
                        "keyword": kw,
                        "content": text[:250],
                        "url": l["href"],
                        "timestamp": datetime.now().isoformat()
                    })
        except Exception as e:
            print(f"⚠️ Truth Social error: {e}")
        time.sleep(2)
    return results

In [10]:
def fetch_mewe_posts(keywords):
    print("🌐 Fetching MeWe posts (DuckDuckGo)...")
    results = []
    for kw in keywords:
        search_url = f"https://duckduckgo.com/html/?q=site:mewe.com {kw}"
        try:
            resp = requests.get(search_url, headers=HEADERS, timeout=10)
            soup = BeautifulSoup(resp.text, "lxml")
            links = soup.select("a.result__a")
            for l in links[:5]:
                text = l.get_text(" ", strip=True)
                if kw.lower() in text.lower():
                    results.append({
                        "source": "MeWe",
                        "keyword": kw,
                        "content": text[:250],
                        "url": l["href"],
                        "timestamp": datetime.now().isoformat()
                    })
        except Exception as e:
            print(f"⚠️ MeWe error: {e}")
        time.sleep(2)
    return results

In [15]:
all_results = []
for func in [
    fetch_mastodon_posts,
    fetch_twitter_posts,
    fetch_telegram_posts,
    fetch_parler_posts,
    fetch_truthsocial_posts,
    fetch_mewe_posts
]:
    all_results += func(KEYWORDS)


🐘 Fetching Mastodon posts...
🐦 Fetching Twitter (via Nitter)...
⚠️ Twitter error for 'medicine': ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
⚠️ Twitter error for 'medical': ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
⚠️ Twitter error for 'drug': ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
⚠️ Twitter error for 'therapy': ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
⚠️ Twitter error for 'treatment': ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
⚠️ Twitter error for 'breakthrough': ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
⚠️ Twitter error for 'research': ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
⚠️ Twitter error for 'innovation': ('Connection aborted

In [16]:
if all_results:
    df = pd.DataFrame(all_results)
    if os.path.exists(OUTPUT_FILE):
        df.to_csv(OUTPUT_FILE, mode="a", header=False, index=False, encoding="utf-8")
    else:
        df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8")
    print(f"✅ {len(df)} results found and saved to {OUTPUT_FILE}")
else:
    print("⚠️ No keyword hits found this time.")

✅ 3 results found and saved to social_keyword_hits.csv


In [17]:
from google.colab import files
files.download("social_keyword_hits.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>